In [8]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [12]:
client = MistralClient()

In [64]:
model_list = client.list_models()

In [69]:
for model in model_list.data:
    print(model.id)

mistral-medium
mistral-small
mistral-tiny
mistral-embed


In [10]:
MODEL = 'mistral-tiny'
MESSAGES = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

In [11]:
api_params = dict ( model=MODEL, messages=MESSAGES )

In [13]:
chat_response = client.chat( **api_params ) 

In [23]:
print(chat_response.choices[0].message.content)

Determining the "best" French cheese is subjective and depends on personal preference, as there are over 400 types of French cheese, each with unique flavors, textures, and milks sources. Here are some famous and beloved French cheeses that you might enjoy:

1. Roquefort: A blue-veined sheep's milk cheese with a pungent, tangy flavor and a crumbly texture.
2. Camembert: A soft, creamy cow's milk cheese with a white rind and a strong, earthy flavor.
3. Brie de Meaux: A soft, creamy cow's milk cheese with a bloomy rind and a mild, buttery flavor.
4. Comté: A hard, nutty cow's milk cheese with a delicate, sweet flavor and a firm, granular texture.
5. Munster: A soft, pungent cow's milk cheese with a reddish-orange rind and a strong, smelly flavor.
6. Reblochon: A soft, creamy cow's milk cheese with a bloomy rind and a mild, rich flavor.
7. Époisses: A soft, pungent cow's milk cheese with a reddish-orange rind and a strong, savory flavor.

Ultimately, the best French cheese is the one that

In [24]:
chat_response.usage

UsageInfo(prompt_tokens=15, total_tokens=344, completion_tokens=329)

## Streaming

In [35]:
full_response = ""
for chunk in client.chat_stream(model=MODEL, messages=MESSAGES):
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)
    full_response += content

Determining the "best" French cheese can be subjective as it depends on personal preferences, as there are over 400 types of French cheese, each with unique textures, flavors, and milk sources. Here are some popular and highly regarded French cheeses:

1. Roquefort: A blue-veined cheese from the Massif Central region known for its strong, pungent flavor and creamy texture.
2. Comté: A nutty and slightly sweet cheese from the Franche-Comté region, made from unpasteurized milk using the traditional method.
3. Camembert: A soft, white-rinded cheese with a strong, earthy flavor and runny texture, hailing from Normandy.
4. Brie de Meaux: A popular soft, bloomy-rinded cheese from the Ile-de-France region, known for its mild, buttery flavor and creamy texture.
5. Munster: A pungent, smelly, and spicy cheese from the Alsace region, with a soft, runny texture.
6. Reblochon: A soft, bloomy-rinded cheese from the Savoie region, known for its strong aroma and rich, creamy taste.
7. Chèvre: Goat ch

In [33]:
chunk

ChatCompletionStreamResponse(id='cmpl-6b759adaac9c4682b1701251462ceba5', model='mistral-tiny', choices=[ChatCompletionResponseStreamChoice(index=0, delta=DeltaMessage(role=None, content=''), finish_reason=<FinishReason.stop: 'stop'>)], created=1704204962, object='chat.completion.chunk', usage=None)

## Track tokens

In [51]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

def n_tokens_from_str( string ):
    return len( tokenizer.encode(string) )

In [53]:
chat_response = client.chat( **api_params ) 
n = n_tokens_from_str(chat_response.choices[0].message.content)

In [54]:
n, chat_response.usage.completion_tokens

(495, 494)

In [62]:
input_messages = [
    [
        ChatMessage(role="user", content="What is the best French cheese?"),
    ],
    [
        ChatMessage(role="user", content="What is the best French cheese?"),
        ChatMessage(role="assistant", content="You know, France isn't the only place with excellent cheeses."),
        ChatMessage(role="user", content="What is the best Wisconsin cheese?"),
    ],
    [
        ChatMessage(role="user", content="How is Wisconsin different from France?"),
        ChatMessage(role="assistant", content="Wisconsin is a state in the United States. Frace is a country in Europe."),
        ChatMessage(role="user", content="Who is fancier, Parisiennes or Wisconsines?"),
        ChatMessage(role="assistant", content="The French are fancier. Wisconsinites are decidedly less fancy."),
        ChatMessage(role="user", content="What is similar about them?"),
    ],
]

for msgs in input_messages:

    # Couldn't find any examples on this, this is empirical, and subject to change at any time.
    N_TOKENS_PER_MESSAGE = 3
    TOKEN_BUFFER = 4

    chat_response = client.chat( model=MODEL, messages=msgs )
    n = TOKEN_BUFFER
    for msg in msgs:
        n += n_tokens_from_str(msg.content) + N_TOKENS_PER_MESSAGE
    print(n, chat_response.usage.prompt_tokens)

15 15
46 46
85 85
